# உரை வகைப்படுத்தல் பணிகள்

நாம் முன்பு குறிப்பிட்டபடி, **AG_NEWS** தரவுத்தொகுப்பை அடிப்படையாகக் கொண்டு எளிய உரை வகைப்படுத்தல் பணிகளில் கவனம் செலுத்துவோம். இது செய்தி தலைப்புகளை உலகம், விளையாட்டு, வணிகம் மற்றும் அறிவியல்/தொழில்நுட்பம் ஆகிய 4 வகைகளில் ஒன்றாக வகைப்படுத்துவது ஆகும்.

## தரவுத்தொகுப்பு

இந்த தரவுத்தொகுப்பு [`torchtext`](https://github.com/pytorch/text) மொடியுள் உள்ளடக்கப்பட்டுள்ளது, எனவே இதை எளிதாக அணுகலாம்.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

இங்கே, `train_dataset` மற்றும் `test_dataset` ஆகியவை வரிசைகளைக் கொண்டுள்ளன, அவை முறையே வகுப்பின் எண் (label) மற்றும் உரை (text) ஆகியவற்றின் ஜோடிகளைத் திருப்பி வழங்குகின்றன, உதாரணமாக:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

அப்படியெனில், நமது தரவுத்தொகுப்பில் இருந்து முதல் 10 புதிய தலைப்புகளை அச்சிடுவோம்:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

கணக்கெடுப்புகள் மீள்பார்வையாளர்கள் என்பதால், தரவுகளை பலமுறை பயன்படுத்த விரும்பினால், அதை பட்டியலாக மாற்ற வேண்டும்:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## டோக்கனீகরণ

இப்போது நாங்கள் உரையை **எண்களாக** மாற்ற வேண்டும், அவை டென்சர்களாக பிரதிநிதித்துவப்படுத்தப்படலாம். நாங்கள் சொல் மட்டத்திலான பிரதிநிதித்துவத்தை விரும்பினால், இரண்டு விஷயங்களை செய்ய வேண்டும்:
* உரையை **டோக்கன்களாக** பிரிக்க **டோக்கனைசரை** பயன்படுத்தவும்  
* அந்த டோக்கன்களின் ஒரு **வார்த்தகோஷத்தை** உருவாக்கவும்.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

வார்த்தைக்களஞ்சியத்தைப் பயன்படுத்தி, நமது டோக்கனைக்களிக்கப்பட்ட சரத்தை எளிதாக எண்களின் தொகுப்பாக குறியாக்கலாம்:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Bag of Words உரை பிரதிநிதித்துவம்

சொற்கள் அர்த்தத்தை பிரதிநிதித்துவப்படுத்துவதால், சில நேரங்களில் ஒரு உரையின் அர்த்தத்தை அதன் வாக்கியத்தில் உள்ள சொற்களின் வரிசையைப் பொருட்படுத்தாமல், தனிப்பட்ட சொற்களைப் பார்த்து கண்டறிய முடியும். உதாரணமாக, செய்திகளை வகைப்படுத்தும்போது, *வானிலை*, *பனி* போன்ற சொற்கள் *வானிலை முன்னறிவிப்பு* என்பதை சுட்டிக்காட்டும், அதே நேரத்தில் *பங்கு*, *டாலர்* போன்ற சொற்கள் *நிதி செய்திகள்* என்பதற்காக கணக்கில் எடுத்துக்கொள்ளப்படும்.

**Bag of Words** (BoW) வெக்டர் பிரதிநிதித்துவம் என்பது மிகவும் பரவலாக பயன்படுத்தப்படும் பாரம்பரிய வெக்டர் பிரதிநிதித்துவமாகும். ஒவ்வொரு சொல்லும் ஒரு வெக்டர் குறியீட்டுடன் இணைக்கப்பட்டிருக்கும், வெக்டர் கூறு குறிப்பிட்ட ஆவணத்தில் ஒரு சொல்லின் நிகழ்வுகளின் எண்ணிக்கையை கொண்டிருக்கும்.

![Bag of Words வெக்டர் பிரதிநிதித்துவம் நினைவகத்தில் எப்படி பிரதிநிதித்துவப்படுத்தப்படுகிறது என்பதை காட்டும் படம்.](../../../../../translated_images/ta/bag-of-words-example.606fc1738f1d7ba9.webp)

> **Note**: BoW ஐ உரையில் உள்ள தனிப்பட்ட சொற்களுக்கான அனைத்து ஒரே-ஹாட்-கோடிடப்பட்ட வெக்டர்களின் கூட்டமாகவும் நீங்கள் சிந்திக்கலாம்.

கீழே Scikit Learn பைதான் நூலகத்தைப் பயன்படுத்தி Bag of Words பிரதிநிதித்துவத்தை உருவாக்குவதற்கான ஒரு உதாரணம்:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

எங்கள் AG_NEWS தரவுத்தொகுப்பின் வெக்டர் பிரதிநிதித்துவத்திலிருந்து bag-of-words வெக்டரை கணக்கிட, கீழ்கண்ட செயல்பாட்டைப் பயன்படுத்தலாம்:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **குறிப்பு:** இங்கு நாம் `vocab_size` என்ற உலகளாவிய மாறியை பயன்படுத்தி சொற்கள்தொகையின் இயல்பான அளவை குறிப்பிடுகிறோம். பொதுவாக சொற்கள்தொகையின் அளவு மிகவும் பெரியதாக இருக்கும், எனவே நாம் சொற்கள்தொகையின் அளவை மிகவும் அடிக்கடி பயன்படுத்தப்படும் சொற்களுக்குத் தாழ்த்தலாம். `vocab_size` மதிப்பை குறைத்து கீழே உள்ள குறியீட்டை இயக்கி பாருங்கள், இது துல்லியத்தை எப்படி பாதிக்கிறது என்பதை கவனிக்கவும். நீங்கள் சில துல்லியத்தளர்வை எதிர்பார்க்கலாம், ஆனால் அதிக செயல்திறனுக்காக அது மிகுந்ததாக இருக்காது.


## BoW வகைப்பாட்டாளரை பயிற்சி செய்யுதல்

நாம் எவ்வாறு Bag-of-Words பிரதிநிதித்துவத்தை உருவாக்குவது என்பதை கற்றுக்கொண்ட பிறகு, அதன் மீது ஒரு வகைப்பாட்டாளரை பயிற்சி செய்யலாம். முதலில், positional vector பிரதிநிதித்துவங்கள் அனைத்தும் Bag-of-Words பிரதிநிதித்துவமாக மாற்றப்படும் வகையில், பயிற்சிக்கான எங்கள் தரவுத்தொகுப்பை மாற்ற வேண்டும். இதை சாதாரண torch `DataLoader` இல் `collate_fn` அளவுருவிற்கு `bowify` செயல்பாட்டை அனுப்புவதன் மூலம் அடையலாம்:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

இப்போது, ஒரு எளிய வகைப்பாட்டாளர் நரம்பியல் வலைப்பின்னலை வரையறுப்போம், இது ஒரு நேரியல் அடுக்கு கொண்டுள்ளது. உள்ளீட்டு வெக்டர் அளவு `vocab_size`-க்கு சமமாகும், மற்றும் வெளியீட்டு அளவு வகைகளின் எண்ணிக்கையை (4) பிரதிநிதித்துவப்படுத்துகிறது. நாம் வகைப்பாடு பணியை தீர்க்கிறோம் என்பதால், இறுதி செயல்பாட்டு செயல்பாடு `LogSoftmax()` ஆகும்.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

இப்போது நாங்கள் ஒரு நிலையான PyTorch பயிற்சி மடக்கு வரிசையை வரையறுக்கப் போகிறோம். எங்கள் தரவுத்தொகுப்பு மிகவும் பெரியதாக இருப்பதால், எங்கள் கற்பித்தல் நோக்கத்திற்காக நாங்கள் ஒரு எபோக்கிற்கே பயிற்சி செய்யப் போகிறோம், சில நேரங்களில் ஒரு எபோக்கிற்கும் குறைவாக பயிற்சி செய்யலாம் (`epoch_size` அளவுருவை குறிப்பிடுவதன் மூலம் பயிற்சியை வரையறுக்க முடியும்). பயிற்சியின் போது சேர்க்கப்பட்ட பயிற்சி துல்லியத்தையும் நாங்கள் அறிக்கையிடுவோம்; அறிக்கையிடும் அடிக்கடி தன்மையை `report_freq` அளவுருவை பயன்படுத்தி குறிப்பிடலாம்.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams மற்றும் N-Grams

ஒரு bag of words அணுகுமுறையின் ஒரு வரம்பு என்னவென்றால், சில சொற்கள் பல சொற்கள் கொண்ட வெளிப்பாடுகளின் ஒரு பகுதியாக இருக்கின்றன. உதாரணமாக, 'hot dog' என்ற சொல் மற்ற சூழல்களில் 'hot' மற்றும் 'dog' என்ற சொற்களைவிட முற்றிலும் வேறுபட்ட அர்த்தம் கொண்டது. 'hot' மற்றும் 'dog' என்ற சொற்களை எப்போதும் ஒரே வெக்டர்களால் பிரதிநிதித்துவப்படுத்தினால், அது எங்கள் மாதிரியை குழப்பமடையச் செய்யலாம்.

இதற்கு தீர்வாக, **N-gram பிரதிநிதித்துவங்கள்** ஆவண வகைப்பாட்டின் முறைகளில் அடிக்கடி பயன்படுத்தப்படுகின்றன, இங்கு ஒவ்வொரு சொல், இரு சொற்கள் அல்லது மூன்று சொற்களின் அடர்த்தி வகைப்பாட்டாளர்களை பயிற்சி செய்ய பயனுள்ள அம்சமாக இருக்கும். உதாரணமாக, bigram பிரதிநிதித்துவத்தில், அசல் சொற்களுடன் சேர்த்து அனைத்து சொல் ஜோடிகளையும் சொற்களஞ்சியத்தில் சேர்ப்போம்.

கீழே Scikit Learn பயன்படுத்தி bigram bag of word பிரதிநிதித்துவத்தை உருவாக்குவதற்கான ஒரு உதாரணம் உள்ளது:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram முறையின் முக்கிய குறைபாடு என்னவென்றால், சொற்களஞ்சியத்தின் அளவு மிகவும் வேகமாக அதிகரிக்கத் தொடங்குகிறது. நடைமுறையில், N-gram பிரதிநிதித்துவத்தை *embedding* போன்ற சில பரிமாணக் குறைப்புத் தொழில்நுட்பங்களுடன் இணைக்க வேண்டும், இதைப் பற்றி நாம் அடுத்த பிரிவில் விவாதிக்கப் போகிறோம்.

**AG News** தரவுத்தொகுப்பில் N-gram பிரதிநிதித்துவத்தை பயன்படுத்த, நாங்கள் சிறப்பு ngram சொற்களஞ்சியத்தை உருவாக்க வேண்டும்:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


நாம் மேலே உள்ள அதே குறியீட்டை பயன்படுத்தி வகைப்பாட்டாளரை பயிற்சி செய்யலாம், ஆனால் இது நினைவகத்தை மிகவும் வீணாக்கும். அடுத்த பிரிவில், நாங்கள் எம்பெடிங்குகளைப் பயன்படுத்தி பைகிராம் வகைப்பாட்டாளரை பயிற்சி செய்யப் போகிறோம்.

> **குறிப்பு:** நீங்கள் குறிப்பிட்ட எண்ணிக்கையை விட அதிகமாக உரையில் தோன்றும் ngrams-களை மட்டும் விடலாம். இது அரிதாக தோன்றும் பைகிராம்களை நீக்குவதற்கு உதவுகிறது மற்றும் பரிமாணத்தை குறிப்பிடத்தக்க அளவில் குறைக்கும். இதைச் செய்ய, `min_freq` அளவுருவை அதிக மதிப்புக்கு அமைக்கவும், மற்றும் சொற்கள்தொகை நீள மாற்றத்தை கவனிக்கவும்.


## வார்த்தை அதிர்வெண்-மாறாக ஆவண அதிர்வெண் (TF-IDF)

BoW (Bag of Words) பிரதிநிதித்துவத்தில், வார்த்தை தோற்றங்கள் வார்த்தையின் தன்மையைப் பொருட்படுத்தாமல் சமமாக மதிப்பீடு செய்யப்படுகின்றன. ஆனால், *a*, *in* போன்ற பொதுவான வார்த்தைகள் வகைப்பாட்டிற்குப் பெரிதும் முக்கியமில்லாதவை, அதேசமயம் சிறப்பு வார்த்தைகள் முக்கியமானவை. உண்மையில், பெரும்பாலான இயற்கை மொழி செயலாக்க பணிகளில் சில வார்த்தைகள் மற்றவற்றைவிட அதிகம் தொடர்புடையவை.

**TF-IDF** என்பது **term frequency–inverse document frequency** என்பதற்கான சுருக்கமாகும். இது Bag of Words முறைமையின் ஒரு மாறுபாடு, இதில் ஒரு ஆவணத்தில் ஒரு வார்த்தை தோன்றியதைக் குறிக்கும் 0/1 மதிப்பின் பதிலாக, ஒரு மிதவை புள்ளி மதிப்பு பயன்படுத்தப்படுகிறது, இது அந்த வார்த்தையின் தொகுப்பில் தோன்றும் அதிர்வெண் தொடர்புடையதாக இருக்கும்.

முறைமையாக, ஒரு ஆவணத்தில் $i$ என்ற வார்த்தையின் $w_{ij}$ என்ற எடை கீழே கொடுக்கப்பட்டுள்ளதுபோல் வரையறுக்கப்படுகிறது:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
இதில்
* $tf_{ij}$ என்பது $j$ ஆவணத்தில் $i$ வார்த்தை தோன்றிய எண்ணிக்கை, அதாவது நாம் முன்பு பார்த்த BoW மதிப்பு
* $N$ என்பது தொகுப்பில் உள்ள ஆவணங்களின் எண்ணிக்கை
* $df_i$ என்பது முழு தொகுப்பில் $i$ வார்த்தை உள்ள ஆவணங்களின் எண்ணிக்கை

TF-IDF மதிப்பு $w_{ij}$ ஒரு ஆவணத்தில் ஒரு வார்த்தை தோன்றும் எண்ணிக்கைக்கு நேர்மறையாக அதிகரிக்கிறது, மேலும் அந்த வார்த்தை தொகுப்பில் உள்ள ஆவணங்களின் எண்ணிக்கையால் சமநிலைப்படுத்தப்படுகிறது, இது சில வார்த்தைகள் மற்றவற்றைவிட அதிகமாக தோன்றும் உண்மையை சரிசெய்ய உதவுகிறது. உதாரணமாக, ஒரு வார்த்தை தொகுப்பில் உள்ள *ஒவ்வொரு* ஆவணத்திலும் தோன்றினால், $df_i=N$, மற்றும் $w_{ij}=0$, மற்றும் அந்த வார்த்தைகள் முற்றிலும் புறக்கணிக்கப்படும்.

Scikit Learn பயன்படுத்தி TF-IDF வகைபடுத்தலை எளிதாக உருவாக்கலாம்:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## முடிவு

TF-IDF பிரதிநிதிகள் வெவ்வேறு சொற்களுக்கு அதிர்வெண் எடையை வழங்கினாலும், அவை அர்த்தத்தை அல்லது வரிசையை பிரதிநிதித்துவப்படுத்த முடியாது. பிரபலமான மொழியியல் நிபுணர் ஜே. ஆர். ஃபிர்த் 1935 ஆம் ஆண்டில் கூறியதுபோல், “ஒரு சொல்லின் முழுமையான அர்த்தம் எப்போதும் சூழலுடன் தொடர்புடையது, மற்றும் சூழலைத் தவிர்த்து அர்த்தத்தைப் பற்றிய எந்த ஆய்வும் முக்கியத்துவம் பெறாது.”. இந்த பாடத்தில் பின்னர், மொழி மாதிரிகளைப் பயன்படுத்தி உரைமூலமாக சூழலியல் தகவலை எவ்வாறு பிடிக்கலாம் என்பதை நாம் கற்றுக்கொள்வோம்.



---

**அறிவிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையை பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் சொந்த மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்களுக்கும் அல்லது தவறான விளக்கங்களுக்கும் நாங்கள் பொறுப்பல்ல.
